In [1]:
#引入库
import csv
import os
from __future__ import print_function, division
from PIL import Image
import numpy as np
import h5py
import torch.utils.data as data
import argparse
import torch.optim as optim
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
import matplotlib.pyplot as plt
import os
from torchvision import datasets, transforms
import time
from torch.utils.data import DataLoader, Dataset
import torchvision
#使用显卡
torch.cuda.is_available()
#显示显卡类型
torch.cuda.get_device_name(0)

'NVIDIA GeForce RTX 4060 Laptop GPU'

In [2]:
Training_x = []
Training_y = []
PublicTest_x = []
PublicTest_y = []
PrivateTest_x = []
PrivateTest_y = []
 
file = r'D:/code/ai/FER/fer2013.csv' #文件路径
 
datapath = os.path.join('data', 'data.h5')   #连接两个或更多的路径名组件
if not os.path.exists(os.path.dirname(datapath)): #os.path.dirname去掉文件名，返回目录
    os.makedirs(os.path.dirname(datapath))
 
with open(file, 'r') as csvin:
    data = csv.reader(csvin)
    for row in data:
        if row[-1] == 'Training':
            temp_list = []
            for pixel in row[1].split( ):
                temp_list.append(int(pixel))  #int() 函数用于将一个字符串或数字转换为整型
            I = np.asarray(temp_list)
            Training_y.append(int(row[0]))  #在列表末尾添加新的对象list.append(erd)
            Training_x.append(I.tolist())   #转化为List集合
 
        if row[-1] == "PublicTest" :
            temp_list = []
            for pixel in row[1].split( ):
                temp_list.append(int(pixel))
            I = np.asarray(temp_list)
            PublicTest_y.append(int(row[0]))
            PublicTest_x.append(I.tolist())
 
        if row[-1] == 'PrivateTest':
            temp_list = []
            for pixel in row[1].split( ):
                temp_list.append(int(pixel))
            I = np.asarray(temp_list)
 
            PrivateTest_y.append(int(row[0]))
            PrivateTest_x.append(I.tolist())
 
print(np.shape(Training_x))
print(np.shape(PublicTest_x))
print(np.shape(PrivateTest_x))
 
datafile = h5py.File(datapath, 'w')
datafile.create_dataset("Training_pixel", dtype = 'uint8', data=Training_x)
datafile.create_dataset("Training_label", dtype = 'int64', data=Training_y)
datafile.create_dataset("PublicTest_pixel", dtype = 'uint8', data=PublicTest_x)
datafile.create_dataset("PublicTest_label", dtype = 'int64', data=PublicTest_y)
datafile.create_dataset("PrivateTest_pixel", dtype = 'uint8', data=PrivateTest_x)
datafile.create_dataset("PrivateTest_label", dtype = 'int64', data=PrivateTest_y)
datafile.close()
 
print("Save data finish!!!")

(28709, 2304)
(3589, 2304)
(3589, 2304)
Save data finish!!!


In [3]:
from __future__ import print_function
from PIL import Image
import numpy as np
import h5py
import torch.utils.data as data
 
class FER2013(data.Dataset):
    """`FER2013 Dataset.
    Args:
        train (bool, optional): If True, creates dataset from training set, otherwise
            creates from test set.
        transform (callable, optional): A function/transform that  takes in an PIL image
            and returns a transformed version. E.g, ``transforms.RandomCrop``
    """
 
    #def __init__(self, split='Training', transform=None):
    def __init__(self, split, transform=None):    
        self.transform = transform
        self.split = split  # training set or test set
        self.data = h5py.File('./data/data.h5', 'r', driver='core')
        # now load the picked numpy arrays
        if self.split == 'Training':
            self.train_data = self.data['Training_pixel']
            self.train_labels = self.data['Training_label']
            self.train_data = np.asarray(self.train_data)
            self.train_data = self.train_data.reshape((28709, 48, 48))
            self.train_data = np.array(self.train_data)
           
 
        elif self.split == 'PublicTest':
            self.PublicTest_data = self.data['PublicTest_pixel']
            self.PublicTest_labels = self.data['PublicTest_label']
            self.PublicTest_data = np.asarray(self.PublicTest_data)
            self.PublicTest_data = self.PublicTest_data.reshape((3589, 48, 48))
            self.PublicTest_data = np.array(self.PublicTest_data)
 
        else  :
            self.PrivateTest_data = self.data['PrivateTest_pixel']
            self.PrivateTest_labels = self.data['PrivateTest_label']
            self.PrivateTest_data = np.asarray(self.PrivateTest_data)
            self.PrivateTest_data = self.PrivateTest_data.reshape((3589, 48, 48))
            self.PrivateTest_data = np.array(self.PrivateTest_data)
 
    def __getitem__(self, index):
        """
        Args:
            index (int): Index
        Returns:
            tuple: (image, target) where target is index of the target class.
        """
        if self.split == 'Training':
            img, target = self.train_data[index], self.train_labels[index]
        elif self.split == 'PublicTest':
            img, target = self.PublicTest_data[index], self.PublicTest_labels[index]
        else:
            img, target = self.PrivateTest_data[index], self.PrivateTest_labels[index]
 
        # doing this so that it is consistent with all other datasets
        # to return a PIL Image
        img = img[:, :, np.newaxis]   #  np.newaxis的作用是增加一个维度
        img = np.concatenate((img, img, img), axis=2)  #完成多个数组的拼接
        img = Image.fromarray(img)
        if self.transform is not None:
            img = self.transform(img)
        return img, target
 
    def __len__(self):
        if self.split == 'Training':
            return len(self.train_data)
        elif self.split == 'PublicTest':
            return len(self.PublicTest_data)
        else :
            return len(self.PrivateTest_data)
 
        
print("finish!!!")

finish!!!


In [4]:
#resnet的数据
print('==> Preparing data..')
transform_train = transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
        )])
transform_test = transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
        )])
trainset = FER2013(split = 'Training', transform=transform_train)
trainloader = DataLoader(trainset, batch_size=1 , shuffle=True)#, num_workers=1
PublicTestset = FER2013(split = 'PublicTest', transform=transform_test)
PublicTestloader = DataLoader(PublicTestset, batch_size=1 , shuffle=False)#, num_workers=1)
PrivateTestset = FER2013(split = 'PrivateTest', transform=transform_test)
PrivateTestloader = DataLoader(PrivateTestset, batch_size=1 , shuffle=False)#, num_workers=1)
 
print("finish!!!")

==> Preparing data..
finish!!!


In [5]:
print(len(trainset))
print(len(trainloader))
for batch_datas, batch_labels in trainloader:
    print(batch_datas.size(),batch_labels.size())

28709
28709
torch.Size([1, 3, 224, 224]) torch.Size([1])
torch.Size([1, 3, 224, 224]) torch.Size([1])
torch.Size([1, 3, 224, 224]) torch.Size([1])
torch.Size([1, 3, 224, 224]) torch.Size([1])
torch.Size([1, 3, 224, 224]) torch.Size([1])
torch.Size([1, 3, 224, 224]) torch.Size([1])
torch.Size([1, 3, 224, 224]) torch.Size([1])
torch.Size([1, 3, 224, 224]) torch.Size([1])
torch.Size([1, 3, 224, 224]) torch.Size([1])
torch.Size([1, 3, 224, 224]) torch.Size([1])
torch.Size([1, 3, 224, 224]) torch.Size([1])
torch.Size([1, 3, 224, 224]) torch.Size([1])
torch.Size([1, 3, 224, 224]) torch.Size([1])
torch.Size([1, 3, 224, 224]) torch.Size([1])
torch.Size([1, 3, 224, 224]) torch.Size([1])
torch.Size([1, 3, 224, 224]) torch.Size([1])
torch.Size([1, 3, 224, 224]) torch.Size([1])
torch.Size([1, 3, 224, 224]) torch.Size([1])
torch.Size([1, 3, 224, 224]) torch.Size([1])
torch.Size([1, 3, 224, 224]) torch.Size([1])
torch.Size([1, 3, 224, 224]) torch.Size([1])
torch.Size([1, 3, 224, 224]) torch.Size([1]

In [6]:
model_2 = torchvision.models.resnet18(pretrained=True).eval().cuda()

d:\Anaconda3\envs\f5-tts\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\Anaconda3\envs\f5-tts\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to C:\Users\realw/.cache\torch\hub\checkpoints\resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:03<00:00, 13.1MB/s]


In [7]:
optimizer = torch.optim.Adam(model_2 .parameters(), lr=0.001)   
loss_func = nn.CrossEntropyLoss()  

In [8]:
print(model_2)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [9]:
def accuracy(predictions, labels):
    pred = torch.max(predictions.data, 1)[1]
    rights = pred.eq(labels.data.view_as(pred)).sum()
    return rights, len(labels)

In [16]:
epoch = 2
train_acc = 0
for epoch in range(epoch):
    train_rights = []
    
    for batch_idx, (data, target) in enumerate(trainloader):  
        model_2.train()
        data, target = data.cuda(), target.cuda()
        data = data.type(torch.float32)
        target = target.type(torch.float32)
        data, target = Variable(data, requires_grad=True), Variable(target, requires_grad=True)  # 把数据转换成Variable
        optimizer.zero_grad()  # 优化器梯度初始化为零
        output = model_2(data)  # 把数据输入网络并得到输出，即进行前向传播
        target = target.long()
        loss = loss_func(output, target)  # 交叉熵损失函数
        loss.backward()  # 反向传播梯度
        optimizer.step()  # 结束一次前传+反传之后，更新参数
        right = accuracy(output, target)
        train_rights.append(right)
        
        if batch_idx % 100 == 0:  
            model_2.eval()
            val_rights = []
            for data, target in PrivateTestloader:
                data, target = data.cuda(), target.cuda()
                data = data.type(torch.float32)
                target = target.type(torch.float32)
                data, target = Variable(data, requires_grad=True), Variable(target, requires_grad=True)  # 计算前要把变量变成Variable形式，因为这样子才有梯度
                output = model_2(data)
                right = accuracy(output, target)
                val_rights.append(right)
            
            # 准确率计算
            train_r = (sum([tup[0] for tup in train_rights]), sum([tup[1] for tup in train_rights]))
            val_r = (sum([tup[0] for tup in val_rights]), sum([tup[1] for tup in val_rights]))
            
            print('当前epoch：{} [{}/{} ({:.0f}%)]\t损失: {:.6f}\t训练集准确率:{:.2f}%\t测试集正确率: {:.2f}%'.format(
                epoch, batch_idx * 1, len(trainloader.dataset),
                100. * batch_idx / len(trainloader),
                loss.data,
                100. * train_r[0].item() / train_r[1],
                100. * val_r[0].item() / val_r[1]
            ))

当前epoch：0 [0/28709 (0%)]	损失: 7.349345	训练集准确率:0.00%	测试集正确率: 0.00%
当前epoch：0 [100/28709 (0%)]	损失: 3.034757	训练集准确率:11.88%	测试集正确率: 15.77%
当前epoch：0 [100/28709 (0%)]	损失: 3.034757	训练集准确率:11.88%	测试集正确率: 15.77%
当前epoch：0 [200/28709 (1%)]	损失: 2.781576	训练集准确率:15.92%	测试集正确率: 14.68%
当前epoch：0 [200/28709 (1%)]	损失: 2.781576	训练集准确率:15.92%	测试集正确率: 14.68%
当前epoch：0 [300/28709 (1%)]	损失: 1.453739	训练集准确率:15.95%	测试集正确率: 14.71%
当前epoch：0 [300/28709 (1%)]	损失: 1.453739	训练集准确率:15.95%	测试集正确率: 14.71%
当前epoch：0 [400/28709 (1%)]	损失: 5.440673	训练集准确率:15.96%	测试集正确率: 19.17%
当前epoch：0 [400/28709 (1%)]	损失: 5.440673	训练集准确率:15.96%	测试集正确率: 19.17%
当前epoch：0 [500/28709 (2%)]	损失: 2.518153	训练集准确率:15.77%	测试集正确率: 22.79%
当前epoch：0 [500/28709 (2%)]	损失: 2.518153	训练集准确率:15.77%	测试集正确率: 22.79%
当前epoch：0 [600/28709 (2%)]	损失: 2.085382	训练集准确率:15.64%	测试集正确率: 10.17%
当前epoch：0 [600/28709 (2%)]	损失: 2.085382	训练集准确率:15.64%	测试集正确率: 10.17%
当前epoch：0 [700/28709 (2%)]	损失: 1.454769	训练集准确率:16.55%	测试集正确率: 13.35%
当前epoch：0 [700/28709 (2%)]	损失: 1.45476

KeyboardInterrupt: 

In [17]:
# 保存整个模型
torch.save(model_2, 'model_2.pth')

# 或者只保存模型的参数（推荐方式）
torch.save(model_2.state_dict(), 'model_2_state_dict.pth')

print("模型已保存！")

torch.save({
    'epoch': epoch,
    'model_state_dict': model_2.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'loss': loss
}, 'checkpoint.pth')

'''
model_2 = torchvision.models.resnet18(pretrained=False).cuda()  # 初始化模型
model_2.load_state_dict(torch.load('model_2_state_dict.pth'))
model_2.eval()  # 切换到评估模式

checkpoint = torch.load('checkpoint.pth')
model_2.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']

model_2.train()  # 切换到训练模式
'''

模型已保存！


"\nmodel_2 = torchvision.models.resnet18(pretrained=False).cuda()  # 初始化模型\nmodel_2.load_state_dict(torch.load('model_2_state_dict.pth'))\nmodel_2.eval()  # 切换到评估模式\n\ncheckpoint = torch.load('checkpoint.pth')\nmodel_2.load_state_dict(checkpoint['model_state_dict'])\noptimizer.load_state_dict(checkpoint['optimizer_state_dict'])\nepoch = checkpoint['epoch']\nloss = checkpoint['loss']\n\nmodel_2.train()  # 切换到训练模式\n"